In [1]:
import sys
sys.path.insert(1, "../Generation Automates")

In [2]:
from automata import *

# Génération des séquences de jonglage simple associées à des musiques

## Limitations

Évidemment, les solutions générées ne sont pas trop réalisables car elles demandent vite des lancers d'une hauteur très importante.

## Idée de tout ce qui suit

- Transformer une séquence de jonglage à partir d'un certain état en une musique
- Puis, pour transformer une musique en séquence de jonglage :
    - Pour chaque état de l'automate à b balles et de hauteur maximale h :
        - Parcourir en l'automate paresseusement en partant de cet état et en backtrackant lorsque la musique que l'on est en train de construire n'est diffère de celle fournie.

Avec ce procédé, on construit (je pense) toutes les séquences de jonglage correspondant à la musique voulue. Il faut aussi rajouter, au début de cette séquence, la suite de lancers pour construire l'état nécessaire au début de la séquence.

## Cas du jonglage simple - Uniquement les rythmes

### Séquence -> Musique

In [3]:
a = Automata(3, 5)

In [4]:
def seq_to_music(a, initial, seq):
    s = initial
    tcnt = 0
    l = []
    for throw in seq:
        tcnt += 1
        if s[0]:
            l.append(tcnt)
            tcnt = 0
        s = a.transition(s, throw)
    if s[0]:
        tcnt += 1
        l.append(tcnt)
    return l

In [5]:
seq_test1 = [3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
seq_test2 = [5, 3, 5, 0, 5, 3, 0]

In [6]:
seq_to_music(a, [True, True, True, False, False], seq_test1)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [ ]:
|>   1          1          2                     1          2                     1
|>   1          1          1                     2          1                     2
     X          X          X                     X          X                     X
   11100 -5-> 11001 -3-> 10110 -5-> 01101 -0-> 11010 -5-> 10101 -3-> 01110 -0-> 11100

In [ ]:
(do, 1), (do, 2), (ré, 6), (mi, 9), (do, 14), (ré, 15), (mi, 6)

  1     2   3  4  5  6  7  8   9  10  11  12 13  14   15
  do--1-do-------------------12------------------do
  ----------6--------ré-----------------9-------------ré
  ----------6--------mi----3---mi

In [ ]:
do ---- do ---- do

G       D       G
D       G       D

do^n : do    do    do     ...    do

$x_{D, 1}, x_{G, 1}, ..., x_{D, n}, x_{G, n}$

$\forall i, X_{D,i} ...$  une formule d'unicité

$X_{D_i} <=> X_{G, i + 1}$

$(x \vee y \vee z) \wedge (t \vee \neg x \vee \neg y)$

In [ ]:
[[x, y, z], [t, -x, -y]]

In [ ]:
4 2
1 2 3 0
4 -1 -2 0

In [7]:
seq_to_music(a, [True, True, True, False, False], seq_test2)

[1, 1, 1, 2, 1, 2]

### Musique -> Séquence

In [8]:
def seq_to_initial(seq, max_height):
    seq1 = [e for e in seq]
    for i in range(len(seq1)):
        if i + seq1[i] < len(seq1):
            seq1[i + seq1[i]] = 0
    return [e != 0 for e in seq1][:max_height]

In [9]:
seq_to_initial(seq_test1, 5)

[True, True, True, False, False]

In [10]:
seq_to_initial(seq_test2, 5)

[True, True, True, False, False]

In [11]:
def states(b, h):
    if h == 0:
        return []
    elif h > 0:
        def states_rec(b, i, l):
            if b == 0 and i >= -1:
                yield l.copy()
            elif b > 0 and i >= 0:
                l1 = l.copy()
                l1[i] = True
                for e in states_rec(b - 1, i - 1, l1):
                    yield e
                l1[i] = False
                for e in states_rec(b, i - 1, l1):
                    yield e
        return states_rec(b, h - 1, [False for i in range(h)])

In [12]:
states(3, 0)

[]

In [13]:
def music_to_seq_rec(a, music, possibilities, initial, state, seq):
    m = seq_to_music(a, initial, seq)
    for i in range(len(m)):
        if m[i] != music[i]:
            raise Exception("backtrack")
    if len(m) == len(music):
        initial_str = a.state_str(initial)
        if initial_str in possibilities:
            possibilities[initial_str].append(seq)
        else:
            possibilities[initial_str] = [seq]
        return
    for throw, next_state in a.possible_transitions(state).items():
        try:
            music_to_seq_rec(a, music, possibilities, initial, next_state, seq + [throw])
        except Exception as e:
            if e.args[0] != "backtrack":
                raise e

In [14]:
def music_to_seq(a, music):
    possibilities = dict()
    for initial in states(a.nb_balls, a.max_height):
        try:
            music_to_seq_rec(a, music, possibilities, initial, initial, [])
        except Exception as e:
            if e.args[0] != "backtrack":
                raise e
    return possibilities

In [15]:
music_to_seq(Automata(3, 5), [1, 1, 1, 1, 1])

{'oxxoo': [[1, 1, 3, 3],
  [1, 1, 3, 4],
  [1, 1, 3, 5],
  [1, 1, 4, 2],
  [1, 1, 4, 4],
  [1, 1, 4, 5],
  [1, 1, 5, 2],
  [1, 1, 5, 3],
  [1, 1, 5, 5]],
 'oxoxo': [[1, 2, 3, 3],
  [1, 2, 3, 4],
  [1, 2, 3, 5],
  [1, 2, 4, 2],
  [1, 2, 4, 4],
  [1, 2, 4, 5],
  [1, 2, 5, 2],
  [1, 2, 5, 3],
  [1, 2, 5, 5],
  [1, 4, 1, 3],
  [1, 4, 1, 4],
  [1, 4, 1, 5],
  [1, 5, 1, 2],
  [1, 5, 1, 4],
  [1, 5, 1, 5]],
 'ooxxo': [[2, 2, 3, 3],
  [2, 2, 3, 4],
  [2, 2, 3, 5],
  [2, 2, 4, 2],
  [2, 2, 4, 4],
  [2, 2, 4, 5],
  [2, 2, 5, 2],
  [2, 2, 5, 3],
  [2, 2, 5, 5],
  [2, 4, 1, 3],
  [2, 4, 1, 4],
  [2, 4, 1, 5],
  [2, 5, 1, 2],
  [2, 5, 1, 4],
  [2, 5, 1, 5],
  [3, 1, 3, 3],
  [3, 1, 3, 4],
  [3, 1, 3, 5],
  [3, 1, 4, 2],
  [3, 1, 4, 4],
  [3, 1, 4, 5],
  [3, 1, 5, 2],
  [3, 1, 5, 3],
  [3, 1, 5, 5],
  [5, 1, 1, 3],
  [5, 1, 1, 4],
  [5, 1, 1, 5]],
 'oxoox': [[1, 3, 3, 3],
  [1, 3, 3, 4],
  [1, 3, 3, 5],
  [1, 3, 4, 2],
  [1, 3, 4, 4],
  [1, 3, 4, 5],
  [1, 3, 5, 2],
  [1, 3, 5, 3],
  [1, 3, 5, 5],
 

In [16]:
seq_to_music(a, [True, True, True, False, False], [5, 3, 1, 5])

[1, 1, 1, 1, 1]

In [17]:
music_to_seq(Automata(3, 5), [1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1])

{'oxxoo': [[1, 1, 4, 5, 5, 0, 4, 0, 3, 3, 3, 3, 3],
  [1, 1, 4, 5, 5, 0, 4, 0, 3, 3, 3, 3, 4],
  [1, 1, 4, 5, 5, 0, 4, 0, 3, 3, 3, 3, 5],
  [1, 1, 4, 5, 5, 0, 4, 0, 3, 3, 3, 4, 2],
  [1, 1, 4, 5, 5, 0, 4, 0, 3, 3, 3, 4, 4],
  [1, 1, 4, 5, 5, 0, 4, 0, 3, 3, 3, 4, 5],
  [1, 1, 4, 5, 5, 0, 4, 0, 3, 3, 3, 5, 2],
  [1, 1, 4, 5, 5, 0, 4, 0, 3, 3, 3, 5, 3],
  [1, 1, 4, 5, 5, 0, 4, 0, 3, 3, 3, 5, 5],
  [1, 1, 4, 5, 5, 0, 4, 0, 3, 3, 4, 2, 3],
  [1, 1, 4, 5, 5, 0, 4, 0, 3, 3, 4, 2, 4],
  [1, 1, 4, 5, 5, 0, 4, 0, 3, 3, 4, 2, 5],
  [1, 1, 4, 5, 5, 0, 4, 0, 3, 3, 4, 4, 1],
  [1, 1, 4, 5, 5, 0, 4, 0, 3, 3, 4, 5, 1],
  [1, 1, 4, 5, 5, 0, 4, 0, 3, 3, 5, 2, 2],
  [1, 1, 4, 5, 5, 0, 4, 0, 3, 3, 5, 2, 4],
  [1, 1, 4, 5, 5, 0, 4, 0, 3, 3, 5, 2, 5],
  [1, 1, 4, 5, 5, 0, 4, 0, 3, 3, 5, 3, 1],
  [1, 1, 4, 5, 5, 0, 4, 0, 3, 3, 5, 5, 1],
  [1, 1, 4, 5, 5, 0, 4, 0, 3, 4, 2, 3, 3],
  [1, 1, 4, 5, 5, 0, 4, 0, 3, 4, 2, 3, 4],
  [1, 1, 4, 5, 5, 0, 4, 0, 3, 4, 2, 3, 5],
  [1, 1, 4, 5, 5, 0, 4, 0, 3, 4, 2, 4, 2],
  

In [18]:
def pp_propositions(d):
    for k in d.keys():
        print(k + " : " + str(d[k][0]))
        for i in range(1, len(d[k])):
            print("".join([" " for e in range(len(k) + 3)]) + str(d[k][i]))

In [19]:
a = Automata(3, 4)
props = music_to_seq(a, [1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1])
# pp_propositions(props)
print(sum([len(l) for k, l in props.items()]))

0


In [20]:
seq_to_music(a, [True, True, True, False, False], [3, 3, 4, 5, 5, 0, 4, 0, 3, 3, 5, 2, 4])

[1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1]

In [21]:
a = Automata(3, 6)
props = music_to_seq(a, [1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 4])
pp_propositions(props)

oxxoox : [1, 1, 4, 5, 5, 0, 4, 0, 3, 3, 6, 6, 6, 0, 0, 0]
         [1, 1, 4, 5, 5, 0, 4, 0, 4, 2, 6, 6, 6, 0, 0, 0]
         [1, 1, 4, 5, 5, 0, 5, 0, 2, 3, 6, 6, 6, 0, 0, 0]
         [1, 1, 4, 5, 5, 0, 5, 0, 4, 1, 6, 6, 6, 0, 0, 0]
         [1, 1, 4, 5, 5, 0, 6, 0, 2, 2, 6, 6, 6, 0, 0, 0]
         [1, 1, 4, 5, 5, 0, 6, 0, 3, 1, 6, 6, 6, 0, 0, 0]
         [1, 1, 4, 5, 6, 0, 3, 0, 3, 3, 6, 6, 6, 0, 0, 0]
         [1, 1, 4, 5, 6, 0, 3, 0, 4, 2, 6, 6, 6, 0, 0, 0]
         [1, 1, 4, 5, 6, 0, 5, 0, 1, 3, 6, 6, 6, 0, 0, 0]
         [1, 1, 4, 5, 6, 0, 6, 0, 1, 2, 6, 6, 6, 0, 0, 0]
         [1, 1, 4, 6, 4, 0, 4, 0, 3, 3, 6, 6, 6, 0, 0, 0]
         [1, 1, 4, 6, 4, 0, 4, 0, 4, 2, 6, 6, 6, 0, 0, 0]
         [1, 1, 4, 6, 4, 0, 5, 0, 2, 3, 6, 6, 6, 0, 0, 0]
         [1, 1, 4, 6, 4, 0, 5, 0, 4, 1, 6, 6, 6, 0, 0, 0]
         [1, 1, 4, 6, 4, 0, 6, 0, 2, 2, 6, 6, 6, 0, 0, 0]
         [1, 1, 4, 6, 4, 0, 6, 0, 3, 1, 6, 6, 6, 0, 0, 0]
         [1, 1, 4, 6, 6, 0, 2, 0, 3, 3, 6, 6, 6, 0, 0, 0]
         [1, 1

## Cas du jonglage simple - Rythmes et notes

### Automate

In [22]:
class NotesAutomata():
    def __init__(self, balls, max_height):
        self.max_height = max_height
        self.balls = balls
        self.initial = [balls[i] if i < len(balls) else None for i in range(max_height)]
    
    def transition(self, state, time):
        s = state.copy()
        r = s[0]
        del s[0]
        s.append(None)
        
        if r is not None and time > 0:
            if s[time - 1]:
                raise Exception("conflict")
            else:
                s[time - 1] = r
        elif r is not None:
            raise Exception("conflict")
        elif time > 0:
            raise Exception("conflict")
        return s

    def possible_transitions(self, state):
        transitions = {}

        for time in range(self.max_height + 1):
            try:
                s = self.transition(state, time)
                transitions[time] = s
            except Exception as e:
                if e.args[0] != "conflict":
                    raise e
        
        return transitions

    def state_str(self, s):
        s1 = [x if x is not None else "0" for x in s]
        return "-".join(s1)

### Séquence -> Musique

In [23]:
def seq_to_music2(a, initial, seq):
    s = initial
    tcnt = 0
    l = []
    for throw in seq:
        tcnt += 1
        if s[0] is not None:
            l.append((tcnt, s[0]))
            tcnt = 0
        s = a.transition(s, throw)
    if s[0] is not None:
        tcnt += 1
        l.append((tcnt, s[0]))
    return l

In [24]:
seq_to_music2(NotesAutomata(["do", "ré", "mi"], 5), ["do", "ré", "mi", None, None], [3, 3, 3, 3, 3, 3, 3, 3])

[(1, 'do'),
 (1, 'ré'),
 (1, 'mi'),
 (1, 'do'),
 (1, 'ré'),
 (1, 'mi'),
 (1, 'do'),
 (1, 'ré'),
 (1, 'mi')]

In [25]:
{3, 2, 1} - {2}

{1, 3}

In [26]:
len({3, 2, 1})

3

In [27]:
a = {1, 2, 3}
b = {2}
print(a - b)
print(a)

{1, 3}
{1, 2, 3}


In [28]:
a = {1, 2, 3}
print(a.union({5}))
print(a)

{1, 2, 3, 5}
{1, 2, 3}


In [29]:
a = {1, 2, 3}
a.add(5)
print(a)

{1, 2, 3, 5}


### Musique -> Séquence

In [30]:
def states2(balls, h):
    if h == 0:
        return []
    elif h > 0:
        def states_rec(balls, i, l):
            if len(balls) == 0 and i >= -1:
                yield l.copy()
            elif len(balls) > 0 and i >= 0:
                l1 = l.copy()
                for b in balls.union({None}):
                    l1[i] = b
                    for e in states_rec(balls - {b}, i - 1, l1):
                        yield e
        return states_rec(balls, h - 1, [None for i in range(h)])

In [31]:
list(states2({"do", "ré", "mi"}, 5))

[['do', 'mi', 'ré', None, None],
 ['mi', 'do', 'ré', None, None],
 ['mi', 'ré', 'do', None, None],
 ['ré', 'mi', 'do', None, None],
 ['do', 'ré', 'mi', None, None],
 ['ré', 'do', 'mi', None, None],
 ['do', 'mi', None, 'ré', None],
 ['mi', 'do', None, 'ré', None],
 ['do', None, 'mi', 'ré', None],
 [None, 'do', 'mi', 'ré', None],
 ['mi', None, 'do', 'ré', None],
 [None, 'mi', 'do', 'ré', None],
 ['mi', 'ré', None, 'do', None],
 ['ré', 'mi', None, 'do', None],
 ['mi', None, 'ré', 'do', None],
 [None, 'mi', 'ré', 'do', None],
 ['ré', None, 'mi', 'do', None],
 [None, 'ré', 'mi', 'do', None],
 ['do', 'ré', None, 'mi', None],
 ['ré', 'do', None, 'mi', None],
 ['do', None, 'ré', 'mi', None],
 [None, 'do', 'ré', 'mi', None],
 ['ré', None, 'do', 'mi', None],
 [None, 'ré', 'do', 'mi', None],
 ['do', 'mi', None, None, 'ré'],
 ['mi', 'do', None, None, 'ré'],
 ['do', None, 'mi', None, 'ré'],
 [None, 'do', 'mi', None, 'ré'],
 ['mi', None, 'do', None, 'ré'],
 [None, 'mi', 'do', None, 'ré'],
 ['do', No

In [32]:
def music_to_seq_rec2(a, music, possibilities, initial, state, seq):
    m = seq_to_music2(a, initial, seq)
    for i in range(len(m)):
        if m[i] != music[i]:
            raise Exception("backtrack")
    if len(m) == len(music):
        initial_str = a.state_str(initial)
        if initial_str in possibilities:
            possibilities[initial_str].append(seq)
        else:
            possibilities[initial_str] = [seq]
        return
    for throw, next_state in a.possible_transitions(state).items():
        try:
            music_to_seq_rec2(a, music, possibilities, initial, next_state, seq + [throw])
        except Exception as e:
            if e.args[0] != "backtrack":
                raise e

In [33]:
def music_to_seq2(a, music):
    possibilities = dict()
    for initial in states2(set(a.balls), a.max_height):
        try:
            music_to_seq_rec2(a, music, possibilities, initial, initial, [])
        except Exception as e:
            if e.args[0] != "backtrack":
                raise e
    return possibilities

In [34]:
music_to_seq2(NotesAutomata(["do", "ré", "mi"], 5), [(1, "do"), (1, "ré"), (1, "mi"), (1, "do")])

{'do-ré-mi-0-0': [[3, 3, 3],
  [3, 3, 4],
  [3, 3, 5],
  [3, 4, 2],
  [3, 4, 4],
  [3, 4, 5],
  [3, 5, 2],
  [3, 5, 3],
  [3, 5, 5]]}

In [35]:
music_to_seq2(NotesAutomata(["do", "ré", "mi"], 9), [(2, "do"), (2, "do"), (2, "do"), (2, "ré")])

{'0-do-0-0-0-0-0-ré-mi': [[0, 2, 0, 2, 0, 4, 0],
  [0, 2, 0, 2, 0, 5, 0],
  [0, 2, 0, 2, 0, 6, 0],
  [0, 2, 0, 2, 0, 7, 0],
  [0, 2, 0, 2, 0, 8, 0],
  [0, 2, 0, 2, 0, 9, 0]]}

In [36]:
a = NotesAutomata(["do", "ré", "mi"], 9)

In [37]:
a.transition(["do", "ré", "mi", None, None, None, None, None, None], 4)

['ré', 'mi', None, 'do', None, None, None, None, None]

In [38]:
a.transition(['ré', 'mi', None, 'do', None, None, None, None, False], 9)

['mi', None, 'do', None, None, None, None, False, 'ré']

In [39]:
a.transition(['mi', None, 'do', None, None, None, None, False, 'ré'], 9)

[None, 'do', None, None, None, None, False, 'ré', 'mi']

In [40]:
music_to_seq2(NotesAutomata(["do", "ré", "mi"], 12), [(2, "do"), (2, "do"), (2, "do"), (2, "ré"), (2, "mi"), (4, "ré"), (4, "do"), (2, "mi"), (2, "ré"), (2, "ré"), (2, "do")])

{'0-do-0-0-0-0-0-ré-0-mi-0-0': [[0,
   2,
   0,
   2,
   0,
   12,
   0,
   6,
   0,
   10,
   0,
   0,
   0,
   8,
   0,
   0,
   0,
   8,
   0,
   7,
   0,
   2,
   0,
   4,
   0],
  [0,
   2,
   0,
   2,
   0,
   12,
   0,
   6,
   0,
   10,
   0,
   0,
   0,
   8,
   0,
   0,
   0,
   8,
   0,
   7,
   0,
   2,
   0,
   5,
   0],
  [0,
   2,
   0,
   2,
   0,
   12,
   0,
   6,
   0,
   10,
   0,
   0,
   0,
   8,
   0,
   0,
   0,
   8,
   0,
   7,
   0,
   2,
   0,
   6,
   0],
  [0,
   2,
   0,
   2,
   0,
   12,
   0,
   6,
   0,
   10,
   0,
   0,
   0,
   8,
   0,
   0,
   0,
   8,
   0,
   7,
   0,
   2,
   0,
   7,
   0],
  [0,
   2,
   0,
   2,
   0,
   12,
   0,
   6,
   0,
   10,
   0,
   0,
   0,
   8,
   0,
   0,
   0,
   8,
   0,
   7,
   0,
   2,
   0,
   8,
   0],
  [0,
   2,
   0,
   2,
   0,
   12,
   0,
   6,
   0,
   10,
   0,
   0,
   0,
   8,
   0,
   0,
   0,
   8,
   0,
   7,
   0,
   2,
   0,
   9,
   0],
  [0,
   2,
   0,
   2,
   0,
   12,
   0,
   6,
   

In [41]:
a = NotesAutomata(["do", "ré", "mi"], 12)
s1 = ["do", "ré", "mi"] + [None for i in range(3, 12)]
s2 = a.transition(s1, 4)
s3 = a.transition(s2, 9)
s4 = a.transition(s3, 10)
a.state_str(s4)

'0-do-0-0-0-0-0-ré-0-mi-0-0'